In [1]:
import cv2
import os
import time
import numpy as np
from keras.models import load_model
from keras.utils import img_to_array
from keras.applications.mobilenet_v2 import preprocess_input

In [4]:
model = load_model('major_project.h5')

In [7]:
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
directory = os.path.join(os.getcwd(), 'resumes')

# Initialize a Haar Cascade Classifier for face detection
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Initialize the emotion counts dictionary
emotion_counts = {label: 0 for label in emotion_labels}
emotion_counts['No Face'] = 0

# Initialize a video capture object
cap = cv2.VideoCapture('test.mp4')

while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        # Extract face
        face = frame[y:y + h, x:x + w]

        # Preprocess the face for the model
        face = cv2.resize(face, (48, 48))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face = img_to_array(face)
        face = np.expand_dims(face, axis=0)
        face = preprocess_input(face)

        # Make emotion prediction using the Keras model
        emotion_scores = model.predict(face)[0]
        emotion_label = emotion_labels[np.argmax(emotion_scores)]
        
        # Update emotion counts
        emotion_counts[emotion_label] += 1

    if len(faces) == 0:
        emotion_counts['No Face'] += 1

# Release the video capture object
cap.release()

# Print the emotion counts
for label, count in emotion_counts.items():
    print(f"{label}: {count}")

# Adjust emotion counts if needed
emotion_counts['Angry'] *= 0.2
emotion_counts['Disgust'] *= 0.2
emotion_counts['Fear'] *= 0.2
emotion_counts['Happy'] *= 1.3
emotion_counts['Sad'] *= 0.2
emotion_counts['Neutral'] *= 1
emotion_counts['No Face'] = 0

1/1 [==============================] - 0s 36ms/step
Angry: 8298
Disgust: 0
Fear: 0
Happy: 0
Sad: 0
Surprise: 0
Neutral: 0
No Face: 1106
